# ESTUDO E RESOLUÇÃO DO CICLO DE RANKINE COM MODIFICAÇÕES

---

## Descrição do Problema Proposto

A figura a seguir contém o esquema do ciclo termodinâmico de potência
a ser avaliado neste trabalho:

![problema](./images/esquema-problema.png)

As hipóteses para serem adotadas para o ciclo:

 - Fluido de trabalho: vapor d’água;
 - Perda de carga desprezível nos equipamentos de troca térmica;
 - Operação isentrópica das turbinas e das bombas;
 - Vazão mássica (ṁ ) da corrente 1 fixa em 10.000 kg/h;
 - Saída do condensador (corrente 10): líquido saturado.

![problema](./images/tabela-correntes.png)

a) Determinar a eficiência térmica do sistema modificado proposto;

b) Obter a taxa de trabalho realizado em cada turbina; a taxa de calor
fornecido nas caldeiras; a taxa de trabalho consumida em cada bomba;

c) Determinar a vazão mássica de água de resfriamento empregada no
condensador.

d) Construir o diagrama T x s do sistema.

e) Realizar análise paramétrica no desempenho do ciclo considerando a
variação das pressões de descargas das turbinas; e a temperatura de saída
da caldeira 1; (para esta análise, plote gráficos para demonstrar a influência
desses parâmetros na eficiência térmica do ciclo). Discuta os resultados
obtidos.

In [22]:
# Pacotes necessários para a execução do código
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP

In [23]:
# Tabela onde serão armazenados os dados
colunas = ['Corrente', 'Pressão (kPa)', 'Temperatura (°C)', 'Entalpia (kJ/kg)', 'Entropia (kJ/kg·K)', 'Estado', 'ṁ (kg/h)']
df = pd.DataFrame(columns=colunas)

In [24]:
def obter_propriedades(P, T=None, x=None, s=None):
    """Retorna h (kJ/kg) e s (kJ/kg·K) para um dado estado.

    Args:
        P (float): Pressão (kPa).
        T (float, optional): Temperatura (°C). Defaults to None.
        x (float, optional): Título. Defaults to None.
        s (float, optional): Entropia (kJ/kg·K). Defaults to None.

    Returns:
        tuple: h (kJ/kg) e s (kJ/kg·K).
    """
    # Estado definido por P e T
    if T is not None:
        h = CP.PropsSI('H', 'P', P, 'T', T, 'Water') / 1000  # kJ/kg
        s = CP.PropsSI('S', 'P', P, 'T', T, 'Water') / 1000  # kJ/kg·K
    # Estado definido por P e título
    elif x is not None:
        h = CP.PropsSI('H', 'P', P, 'Q', x, 'Water') / 1000
        s = CP.PropsSI('S', 'P', P, 'Q', x, 'Water') / 1000
    # Estado definido por P e entropia (processo isentrópico)
    elif s is not None:
        try:
            # s em kJ/kg·K → J/kg·K
            h = CP.PropsSI('H', 'P', P, 'S', s*1000, 'Water') / 1000
        except:
            # Calcular título se for mistura
            s_liq = CP.PropsSI('S', 'P', P, 'Q', 0, 'Water') / 1000
            s_vap = CP.PropsSI('S', 'P', P, 'Q', 1, 'Water') / 1000
            x = (s - s_liq) / (s_vap - s_liq)
            h = CP.PropsSI('H', 'P', P, 'Q', x, 'Water') / 1000
    return h, s

In [25]:
# Ponto 1: Saída da Caldeira 1 (15 MPa, 600°C)

P1 = 15000     # kPa
T1 = 600       # °C
h1, s1 = obter_propriedades(P1*1e3, T=T1+273.15)
m_dot1 = 10000 # kg/h

# Adicionar dados à tabela para uso posterior
df.loc[len(df)] = [1, P1, T1, h1, s1, 'Vapor superaquecido', m_dot1]

In [26]:
# Ponto 2: Saída da Turbina 1 (4 MPa, expansão isentrópica)
P2 = 4000  # kPa
m_dot2 = m_dot1
h2, s2 = obter_propriedades(P2*1e3, s=s1)  # s2 = s1
T2 = CP.PropsSI('T', 'P', P2*1e3, 'H', h2*1000, 'Water') - 273.15
df.loc[len(df)] = [2, P2, round(T2,1), h2, s2, 'Mistura' if 0 < (h2 - CP.PropsSI('H', 'P', P2*1e3, 'Q', 0, 'Water')/1000) < (CP.PropsSI('H', 'P', P2*1e3, 'Q', 1, 'Water')/1000 - CP.PropsSI('H', 'P', P2*1e3, 'Q', 0, 'Water')/1000) else 'Vapor', m_dot2]

In [27]:
# Ponto 3:
P3 = P2
m_dot3 = 0.82 * m_dot1
h3 = h2
s3 = s2
T3 = T2
# Estado igual ao Ponto 2 (extração para regeneração)
df.loc[len(df)] = [3, P3, T3 ,h3,s3,'-', m_dot3]

In [28]:
# Ponto 4:
P4 = P3
m_dot4 = 0.18 * m_dot1
T4 = T2
h4 = h2
s4 = s2
# Estado igual ao Ponto 2 (extração para regeneração)
df.loc[len(df)] = [4, P4, T4, h4, s4, '-', m_dot4]

In [29]:
# Ponto 5: Saída da Caldeira 2 (4 MPa, 600°C) - Correção da errata
P5 = 4000  # kPa
T5 = 600   # °C
m_dot5 = m_dot3
h5, s5 = obter_propriedades(P5*1e3, T=T5+273.15)
df.loc[len(df)] = [5, P5, T5, h5, s5, 'Vapor superaquecido', m_dot5]

In [30]:
# Ponto 6: Saída da Turbina 2 (500 kPa, expansão isentrópica)
P6 = 500  # kPa
h6, s6 = obter_propriedades(P6*1e3, s=s5)  # s6 = s5
m_dot6 = m_dot5
T6 = CP.PropsSI('T', 'P', P6*1e3, 'H', h6*1000, 'Water') - 273.15
df.loc[len(df)] = [6, P6, round(T6,1), h6, s6, '-', m_dot6]

In [31]:
# Ponto 7: (igual ao Ponto 6)
P7 = P6  # Mesma pressão
m_dot7 = 0.13 * m_dot6
h7 = h6
s7 = s6
T7 = T6
df.loc[len(df)] = [7, P7, T7, h7, s7, '-', m_dot7]

In [32]:
# Ponto 8:
P8 = P6 # Mesma pressão
m_dot8 = 0.87 * m_dot6
h8 = h6
s8 = s6
T8 = T6
df.loc[len(df)] = [8, P8, T8, h8, s8, '-', m_dot8]

In [33]:
# Ponto 9: Saída da Turbina 2
P9 = 10  # kPa
s9 = s6
h9, _ = obter_propriedades(P9*1e3, s=s9)
m_dot9 = m_dot8
T9 = CP.PropsSI('T', 'P', P9*1e3, 'H', h9*1000, 'Water') - 273.15
df.loc[len(df)] = [9, P9, round(T9,1), h9, s9, 'Vapor úmido', m_dot9]

In [34]:
# Ponto 10: Saída do Condensador
P10 = P9
m_dot10 = m_dot9
h10 = CP.PropsSI('H', 'P', P10*1e3, 'Q', 0, 'Water') / 1000
s10 = CP.PropsSI('S', 'P', P10*1e3, 'Q', 0, 'Water') / 1000
T10 = CP.PropsSI('T', 'P', P10*1e3, 'Q', 0, 'Water') - 273.15
df.loc[len(df)] = [10, P10, round(T10,1), h10, s10, 'Líquido saturado', m_dot10]

In [35]:
# Ponto 11: Saída da Bomba 1
P11 = 500  # kPa (pressão aumentada para regeneração)
h11 = h10 + (P11*1e3 - P10*1e3)/(1000 * 1000)  # Trabalho da bomba simplificado
s11 = s10
m_dot11 = m_dot10
T11 = CP.PropsSI('T', 'P', P11*1e3, 'H', h11*1000, 'Water') - 273.15
df.loc[len(df)] = [11, P11, round(T11,1), h11, s11, 'Líquido comprimido', m_dot11]

In [36]:
# Ponto 12:
P12 = P11
m_dot12 = m_dot11 + m_dot7
h12 = (m_dot11 * h11 + m_dot7 * h7) / m_dot12
s12 = (m_dot11 * s11 + m_dot7 * s7) / m_dot12
T12 = CP.PropsSI('T', 'P', P12*1e3, 'H', h12*1000, 'Water') - 273.15
df.loc[len(df)] = [12, P12, round(T12,1), h12, s12, 'Mistura', m_dot12]

In [37]:
# Ponto 13: (saída da Bomba 2)
P13 = 15000  # kPa
m_dot13 = m_dot12
s13 = s12

# Trabalho da Bomba 2 (processo isentrópico)
# W_bomba = v * ΔP, onde v é o volume específico do líquido
v12 = 1 / CP.PropsSI('D', 'P', P12*1e3, 'H', h12*1000, 'Water')
W_bomba2 = v12 * (P13 - P12)  # Trabalho da bomba em J/kg

# Entalpia do Ponto 13
h13 = h12 + W_bomba2

# Temperatura do Ponto 13
T13 = CP.PropsSI('T', 'P', P13*1e3, 'H', h13*1000, 'Water') - 273.15

df.loc[len(df)] = [13, P13, round(T13,1), h13, s13, 'Líquido comprimido', m_dot13]

In [38]:
# Ponto 14: (saída do trocador de calor)
P14 = P13
h14 = (m_dot13 * h13 + m_dot4 * h4) / (m_dot13 + m_dot4)
s14 = (m_dot13 * s13 + m_dot4 * s4) / (m_dot13 + m_dot4)
T14 = CP.PropsSI('T', 'P', P14*1e3, 'H', h14*1000, 'Water') - 273.15
m_dot14 = (m_dot13 + m_dot4) / 2
df.loc[len(df)] = [14, P14, round(T14,1), h14, s14, 'Mistura', m_dot14]

In [39]:
# Ponto 15: (saída do trocador de calor para bomba 3)
P15 = P13 * 0.2667
m_dot15 = m_dot14
h15 = h14
s15 = s14
T15 = T14
df.loc[len(df)] = [15, P15, T15, h15, s15, 'Líquido Saturado', m_dot15]

In [ ]:
# Ponto 16: (saída da Bomba 3)
P16 = 15000  # kPa
m_dot16 = m_dot15
s16 = s15

# W_bomba = v * ΔP, onde v é o volume específico do líquido
v23 = 1 / CP.PropsSI('D', 'P', P15*1e3, 'H', h15*1000, 'Water')
W_bomba3 = v23 * (P16 - P15)  # Trabalho da bomba em J/kg

# Entalpia do Ponto 16
h16 = h15 + W_bomba3

# Temperatura do Ponto 16
T16 = CP.PropsSI('T', 'P', P16*1e3, 'H', h16*1000, 'Water') - 273.15

df.loc[len(df)] = [16, P16, round(T16,1), h16, s16, 'Líquido comprimido', m_dot16]

In [41]:
# Ponto 17:
P17 = P16
m_dot17  = m_dot16 + m_dot14
h17 = (m_dot16 * h16 + m_dot14 * h14) / m_dot17
s17 = (m_dot16 * s16 + m_dot14 * s14) / m_dot17
T17 = CP.PropsSI('T', 'P', P17*1e3, 'H', h17*1000, 'Water') - 273.15
df.loc[len(df)] = [17, P17, round(T17,1), h17, s17, 'Mistura', m_dot17]
df.set_index('Corrente', inplace=True)
df[['Pressão (kPa)', 'Temperatura (°C)', 'Entalpia (kJ/kg)', 'Entropia (kJ/kg·K)']]

,Pressão (kPa),Temperatura (°C),Entalpia (kJ/kg),Entropia (kJ/kg·K)
Corrente,,,,
1,15000.0,600.000000,3583.131761,6.679590
2,4000.0,374.700000,3153.874543,6.679590
3,4000.0,374.656993,3153.874543,6.679590
4,4000.0,374.656993,3153.874543,6.679590
5,4000.0,600.000000,3674.867523,7.370528
6,500.0,275.400000,3013.589893,7.370528
7,500.0,275.350710,3013.589893,7.370528
8,500.0,275.350710,3013.589893,7.370528
9,10.0,45.800000,2335.617568,7.370528


In [42]:
df[['Pressão (kPa)', 'Temperatura (°C)', 'Entalpia (kJ/kg)', 'Entropia (kJ/kg·K)']].to_excel('TabelaCorrentes-Propriedades.xlsx')